In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from plotly import graph_objects as go

In [18]:
class SyntheticData:
    def __init__(self, w, b, num_train, num_val):
        self.num_train = num_train
        self.num_val = num_val
        self.num = num_train + num_val
        self.w = w
        self.b = b
        self.X = tf.random.normal((self.num, len(w)))
        self.y = self.X @ tf.reshape(self.w, (len(w), 1)) + tf.random.normal((self.num, 1)) + b

In [19]:
sdata = SyntheticData(tf.constant([2.0, 4.0]), tf.constant(2.0), 100, 100)

In [27]:
class LinearRegression:
    def __init__(self, num_inputs, lr):
        self.w = tf.Variable(tf.random.normal((num_inputs, 1)))
        self.b = tf.Variable(tf.zeros(1))
        self.lr = lr
        self.train_errors = []
        self.val_errors = []

    def forward(self, X):
        return X @ self.w + self.b

    def loss(self, y, y_hat):
        return tf.math.reduce_mean((y - y_hat)**2 / 2)

    def update(self, grad_w, grad_b):
        self.w = self.w - self.lr * grad_w
        self.b = self.b - self.lr * grad_b

    def fit(self, data: SyntheticData, batch_size, epochs):
        for i in range(epochs):
            self.train_errors.append(tf.constant(0.0))
            self.val_errors.append(tf.constant(0.0))
            for j in range(0, data.num_train, batch_size):
                ind = list(range(j, min(j + batch_size, data.num_train)))
                X, y = tf.gather(data.X, ind), tf.gather(data.y, ind)
                with tf.GradientTape() as g:
                    g.watch(self.w)
                    g.watch(self.b)
                    loss = self.loss(y, self.forward(X))
                grad_w, grad_b = g.gradient(loss, [self.w, self.b])
                self.update(grad_w, grad_b)
                self.train_errors[-1] += loss
            for j in range(data.num_train, data.num, batch_size):
                ind = list(range(j, min(j + batch_size, data.num)))
                X, y = tf.gather(data.X, ind), tf.gather(data.y, ind)
                loss = self.loss(y, self.forward(X))
                self.val_errors[-1] += loss

In [41]:
model = LinearRegression(len(sdata.w), tf.constant(0.05))
model.fit(sdata, 10, 7)

In [33]:
model.train_errors, model.val_errors

([<tf.Tensor: shape=(), dtype=float32, numpy=63.159348>,
  <tf.Tensor: shape=(), dtype=float32, numpy=25.696241>,
  <tf.Tensor: shape=(), dtype=float32, numpy=12.590399>,
  <tf.Tensor: shape=(), dtype=float32, numpy=7.963903>,
  <tf.Tensor: shape=(), dtype=float32, numpy=6.3140492>,
  <tf.Tensor: shape=(), dtype=float32, numpy=5.718616>,
  <tf.Tensor: shape=(), dtype=float32, numpy=5.50053>],
 [<tf.Tensor: shape=(), dtype=float32, numpy=31.114573>,
  <tf.Tensor: shape=(), dtype=float32, numpy=13.838861>,
  <tf.Tensor: shape=(), dtype=float32, numpy=7.6532507>,
  <tf.Tensor: shape=(), dtype=float32, numpy=5.3879943>,
  <tf.Tensor: shape=(), dtype=float32, numpy=4.5323305>,
  <tf.Tensor: shape=(), dtype=float32, numpy=4.1949987>,
  <tf.Tensor: shape=(), dtype=float32, numpy=4.0542088>])

In [42]:
x = list(range(len(model.train_errors)))
fig = go.Figure()
fig.add_traces(go.Scatter(x=x, y=model.train_errors))
fig.add_traces(go.Scatter(x=x, y=model.val_errors))
fig.update_yaxes(type="log", range=[0, 2])

In [43]:
model.w, model.b

(<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
 array([[1.9534918],
        [3.93575  ]], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.9702101], dtype=float32)>)